# Analise exploratória

In [1]:
import pandas as pd

import yake
import spacy

from spacytextblob.spacytextblob import SpacyTextBlob


In [2]:
spacy.__version__

'3.2.4'

In [3]:
data = pd.read_csv('../data/scraping_data.csv.gz', compression='gzip')

In [4]:
data.head()

,Unnamed: 0,city,title,content,year,tstamp,link
0,0,Lisboa,POL | Local Lisboa,POL | Local Lisboa SECÇÕES 1ª Página Destaque ...,1999,19991111042737,https://arquivo.pt/wayback/19991111042737/http...
1,1,Lisboa,JN Editorial - Text57,JN Editorial - Text57 26 milhões para dar casa...,1999,19990822002536,https://arquivo.pt/wayback/19990822002536/http...
2,2,Lisboa,Outras Paginas,Outras Paginas 11 de Novembro de 1999 Igreja d...,1999,19991117215651,https://arquivo.pt/wayback/19991117215651/http...
3,3,Lisboa,JN Editorial - Texult1,JN Editorial - Texult1 Macau: Rão Kyao é o aut...,1999,19991118004529,https://arquivo.pt/wayback/19991118004529/http...
4,4,Lisboa,PÚBLICONLINE-Os Destaques da Primeira Página,PÚBLICONLINE-Os Destaques da Primeira Página S...,1999,19991012235908,https://arquivo.pt/wayback/19991012235908/http...


In [5]:
data.city.value_counts()

Lisboa              5363
Porto               3835
Coimbra             2080
Braga               2047
Guarda              1969
Setúbal             1792
Aveiro              1471
Beja                1403
Leiria              1344
Bragança            1215
Faro                1208
Viseu               1188
Vila Real           1104
Santarém            1072
Castelo Branco       958
Viana do Castelo     943
Évora                908
Portalegre           859
Name: city, dtype: int64

In [6]:
data.groupby(['year', 'city']).size()

year  city            
1999  Aveiro               4
      Beja                 3
      Braga                4
      Bragança             6
      Castelo Branco       3
                          ..
2021  Setúbal             47
      Viana do Castelo    34
      Vila Real           40
      Viseu               36
      Évora               50
Length: 336, dtype: int64

## Keyword Detection

In [7]:
data.content.iloc[1]

'JN Editorial - Text57 26 milhões para dar casa a dez mil Ana Fonseca "É impressionante a obra que a Câmara Municipal de Lisboa está a desenvolver na vertente habitacional" disse, ontem, o ministro João Cravinho, durante a cerimónia de assinatura do 3º contrato do Programa Especial de Realojamento (PER), que contou com a presença do Primeiro Ministro. Na sua intervenção, após a assinatura do documento pelos presidentes da edilidade lisboeta e do Instituto Nacional da Habitação, o ministro do Equipamento, Planeamento e Administração do Território adiantou que "o Município de Lisboa está a combater o flagelo da exclusão social, opondo-lhe com lucidez e inteligência a requalificação urbana". Cravinho fez igualmente questão de salientar que foi a secretária de Estado da Habitação, Leonor Coutinho, "que relançou a política da habitação começando pelo lado mais ingrato, nomeadamente, o de afinamento dos regulamentos existentes e na eliminação dos obstáculos". Numa tenda improvisada no Alto d

In [8]:
language = "pt"
max_ngram_size = 3
deduplication_thresold = 0.9
deduplication_algo = 'seqm'
windowSize = 1
numOfKeywords = 20

custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_thresold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
keywords = custom_kw_extractor.extract_keywords(data.content.iloc[1])

for kw in keywords:
    print(kw)

('Ana Fonseca', 0.0038407678490116988)
('Câmara Municipal', 0.003978555269459013)
('Programa Especial', 0.003978555269459013)
('ministro João Cravinho', 0.008668324735027152)
('Municipal de Lisboa', 0.01572009438684224)
('Habitação', 0.028366783262467544)
('Nacional da Habitação', 0.028753077992326757)
('desenvolver na vertente', 0.030171465142072706)
('Lisboa', 0.030715864992870243)
('António Guterres', 0.032833672166677284)
('Instituto Nacional', 0.03360846799203783)
('Planeamento e Administração', 0.03596424370203167)
('Administração do Território', 0.03711750557651457)
('contrato do Programa', 0.037535659382879095)
('vereador António Abreu', 0.03980619685567849)
('ministro João', 0.0411577149827416)
('ministro', 0.0425564171335201)
('António Guterres enalteceu', 0.047508223170825474)
('Município de Lisboa', 0.04757942103361036)
('Especial de Realojamento', 0.04932804478431916)


## Sentiment Analysis

In [10]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [16]:
nlp = spacy.load("pt_core_news_lg")
nlp.add_pipe('spacytextblob')
#text = "I had a really horrible day. It was the worst day ever! But every now and then I have a really good day that makes me happy."
text = data.content.iloc[1]
doc = nlp(text)
doc._.blob.polarity                            
#doc._.blob.subjectivity                        
doc._.blob.sentiment_assessments
#doc._.blob.ngrams()

Sentiment(polarity=0.01, subjectivity=0.34500000000000003, assessments=[(['social'], 0.03333333333333333, 0.06666666666666667, None), (['local'], 0.0, 0.0, None), (['no', 'total'], -0.0, 0.75, None), (['social'], 0.03333333333333333, 0.06666666666666667, None), (['total'], 0.0, 0.75, None), (['total'], 0.0, 0.75, None), (['social'], 0.03333333333333333, 0.06666666666666667, None), (['total'], 0.0, 0.75, None), (['central'], 0.0, 0.25, None), (['local'], 0.0, 0.0, None)])